In [479]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score

Dataset loading

In [601]:
ds = np.array([[]]).reshape(0,11)
for day in range(9,14,1):
    impact_fname = '../data/PeMS/Incidents/logit/impact_2017_10_{:02d}.csv'
    ds = np.append(ds, np.loadtxt(impact_fname.format(day), delimiter=','), axis=0)

ds = ds[~np.isnan(ds[:,-3])]

onehot_enc = OneHotEncoder(categorical_features=[0,1])
onehot_enc.fit(ds[:,2:4])
encoded = onehot_enc.transform(ds[:,2:4]).toarray()

scaler = StandardScaler()
scaler.fit(ds[:,[4,8,9]])
scaled = scaler.transform(ds[:,[4,8,9]])

ds = np.concatenate((ds[:,:2], encoded, scaled[:,:1], ds[:,5:8], scaled[:,1:], ds[:,-1:]), axis=1)

del scaled, encoded

Model training

In [602]:
logit = LogisticRegressionCV(class_weight={0:.05, 1:.9}, penalty='l2', n_jobs=3)

In [603]:
X_train, X_test, y_train, y_test = train_test_split(ds[:,:-1], ds[:,-1], test_size=.2, random_state=41)

In [604]:
sum(ds[:,-1]==1)/ds.shape[0]

0.049648309559178851

In [653]:
logit = LogisticRegression(class_weight={0:1, 1:20}, penalty='l2', n_jobs=3, tol=1e-10)

In [654]:
logit.fit(X_train[:,2:-1], y_train)

LogisticRegression(C=1.0, class_weight={0: 1, 1: 20}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=3, penalty='l2', random_state=None,
          solver='liblinear', tol=1e-10, verbose=0, warm_start=False)

In [655]:
logit.score(X_test[:,2:-1], y_test)

0.75328859060402686

In [656]:
predicted = logit.predict(X_test[:,2:-1])

In [657]:
sum((y_test==predicted) & (y_test==1))

393

In [658]:
sum(predicted==1)

3013

In [659]:
sum((y_test!=predicted) & (y_test==1))

137

In [660]:
f1_score(y_test, predicted)

0.22184589331075358

In [661]:
recall_score(y_test, predicted)

0.7415094339622641

In [662]:
precision_score(y_test, predicted)

0.13043478260869565

In [641]:
proba = logit.predict_proba(X_test[:,2:-1])

In [642]:
roc_auc_score(y_test, proba[:,1])

0.81827060272782881

In [535]:
X_test.shape

(16063, 92)

In [663]:
fn = X_test[:,:2][(y_test!=predicted) & (y_test==1)]

In [664]:
np.savetxt(fname='../data/PeMS/Incidents/logit/result/fn.csv', X=fn, delimiter=',')

In [665]:
tp = X_test[:,:2][(y_test==predicted) & (y_test==1)]

In [666]:
np.savetxt(fname='../data/PeMS/Incidents/logit/result/tp.csv', X=tp, delimiter=',')